In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [ ]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [ ]:
from core.agent import *
from core.world import * 
from core.render import * 
from core.skill import * 
from core.models import *
from core.message import *

In [ ]:
DEVICE = "cuda"
BELIEF_DIMS = 5


In [ ]:
from sar.sar_agent import *
from sar.sar_world import *
from sar.sar_env_params import MAX_TIME_STEPS
from sar.sar_traits_sampler import *

belief_initializer = SARBeliefInitializer(BELIEF_DIMS)
trait_sampler = SARTraitSampler()

def initialize_swarm(world : BaseWorld):
    swarm = trait_sampler.generate(SWARM_SIZE, DEVICE)
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    swarm = belief_initializer.initialize_beliefs(swarm)

In [ ]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import *
from utils.geo import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)

GEO_TIFF_PATH = "C:\\Users\\Joaquin\\Desktop\\Thesis-Playground\\datasets\\output_SRTMGL1.tif"

def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    geo_map = read_tiff(GEO_TIFF_PATH)[0]
    tx, ty = geo_map.shape


    if tx > world._dims[0] or ty > world._dims[1]:
        start_x = np.random.randint(0, tx - world._dims[0] + 1)
        start_y = np.random.randint(0, ty - world._dims[1] + 1)
        height_map = geo_map[start_x:start_x + world._dims[0], start_y:start_y + world._dims[1]]
        minimum = height_map.min()
        maximum = height_map.max() 
        terrain_map = TerrainMap(height_map, terrain_map._padding, minimum, maximum)


    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    total_victims = world.get_param("total_victims")
    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims, total_victims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [ ]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *
from solution.encoder_net import *
from solution.decoder_net import *
from solution.complex_model import * 

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

In [ ]:
grid_size = 2 * 3 + 1

complex_model = ComplexModel(
    action_dims= 4,
    belief_dims= BELIEF_DIMS,
    grid_size=grid_size, 
    latent_dims= 16, 
    state_dims= 2,
    trait_dims= 3,
    packet_dims= 2, 
)

comms_protocol = SARCommunicationProtocol(complex_model._encoder_net, complex_model._decoder_net)
action_interpreter = SARActionInterpreter(BELIEF_DIMS)

custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter, comms_protocol, time_step_upper= MAX_TIME_STEPS)

complex_model.to(DEVICE)
custom_gym.to(DEVICE)

In [ ]:
custom_gym.reset(42)

### Testing

In [ ]:
parallel_api_test(custom_gym, num_cycles=1_000)
custom_gym.reset()

# Training

In [ ]:

from models.base import * 
from models.idqn import * 
import matplotlib.pyplot as plt

In [ ]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             target_net= complex_model.get_target_net(),
             model= complex_model,
             batch_size=1024,
             device = DEVICE,
             buffer_size= custom_gym._max_time_steps * 4
             )

In [ ]:
rewards = train_loop(custom_gym, model, games=100, optimization_passes = 1, seed=42)

In [ ]:
plt.plot(rewards)

# Optimizations

In [ ]:
import cProfile

In [ ]:
def stress_test():
    train_loop(custom_gym, model, games=1, optimization_passes = 1, seed=42)

In [ ]:

cProfile.run('stress_test()', sort = 'time')

# Visualization

In [ ]:
from models.base import RenderWrapper

renderer = RenderWrapper(model=model)

In [ ]:
renderer.render()